Script to extract GloFAS reanalysis data at station locations stored in an s3 bucket. Metadata file is used to identify which station points to extract (use Lisflood x and y coordinates if available).

In [3]:
import s3fs
import dask
import xarray as xr
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from pathlib import Path

In [6]:
country = 'mozambique'  # define country of interest
#directory = Path(f'/s3/scratch/jamie.towner/flood_aa/{country}')  # define main working directory
directory = Path(r"C:\Users\15133\Documents\WFP\flood_hazard\flood_aa\MOZ_training")  # define main working directory

In [4]:
# Set up the S3 path for the Zarr files
# store = f"s3://wfp-seasmon/input/cds/glofas-historical/saf/01/*.zarr"

# # Set up connection to s3 store
# s3 = s3fs.S3FileSystem.current()

# # Fetch list of .zarr stores (files)
# remote_files = s3.glob(store)
# store = [
#     s3fs.S3Map(root=f"s3://{file}", s3=s3, check=False) for file in remote_files
# ]



In [10]:
store = list(Path.glob(directory / "data/forecasts/glofas_reanalysis/", "*.zarr"))

In [18]:
# Load the CSV file containing station information (i.e., station name, lat, lon)
# define paths to data
station_info = pd.read_csv(directory / "data/metadata/metadata_observations.csv")

# Create the output directory if it doesn't exist
out_dir = directory / "data/forecasts/glofas_reanalysis"
Path(out_dir).mkdir(parents=True, exist_ok=True)

# select only chokwe station for training
station_info = station_info[station_info['station name'] == 'Limpopo_em_Chokwe']

In [19]:
# Initialize a dictionary to store data for each station
station_data = {}

# Initialize tqdm with the total number of iterations to track progress
# total_iterations = len(remote_files) * len(station_info)
# pbar = tqdm(total=total_iterations, desc="Extracting Data")

# Open multiple .zarr files with dask and xarray, setting chunk configuration
with dask.config.set(**{"array.slicing.split_large_chunks": True}):
    ds = xr.open_mfdataset(
        store,
        decode_coords="all",
        engine="zarr",
        parallel=True,  # Enable parallel processing for speed-up
        combine="by_coords"
    )

    # Loop over each station in the station_info CSV
    for index, row in station_info.iterrows():
        point_name = row['station name']
        latitude = row['lisflood_y']
        longitude = row['lisflood_x']
        if np.isnan(latitude) or np.isnan(longitude):
            latitude = row['latitude']
            longitude = row['longitude']

        # Replace 'lat' and 'lon' with 'latitude' and 'longitude'
        lat_index = ds['latitude'].sel(latitude=latitude, method='nearest').values
        lon_index = ds['longitude'].sel(longitude=longitude, method='nearest').values

        # Extract river discharge data for the nearest point
        data_at_point = ds['dis24'].sel(latitude=lat_index, longitude=lon_index).values
        dates = ds.time.values

        # Convert dates to DD/MM/YYYY format
        formatted_dates = pd.to_datetime(dates).strftime('%d/%m/%Y')

        # Create a DataFrame for the extracted data
        extracted_df = pd.DataFrame({'date': formatted_dates, 'river discharge': data_at_point})

        # Append the data to the station's DataFrame within the station_data dictionary
        if point_name not in station_data:
            station_data[point_name] = extracted_df
        else:
            # Merge with the existing data for the same station
            station_data[point_name] = pd.concat([station_data[point_name], extracted_df])
        
        #pbar.update(len(remote_files))  # Update tqdm progress by number of files processed

# Close the tqdm progress bar
#pbar.close()

# Save extracted data for each station to CSV files
for station, data in station_data.items():
    csv_file_name = os.path.join(out_dir, f"{station}.csv")
    data.to_csv(csv_file_name, index=False)

In [20]:
all_dfs = []
for station, data in station_data.items():
    name = "".join(c for c in station if c.isalnum() or c in (' ', '_')).replace(' ', '_')
    data = data.rename(columns={'river discharge':name})
    data = data.set_index('date')
    all_dfs.append(data)
pd.concat(all_dfs,axis=1)

,Limpopo_em_Chokwe
date,
01/01/1979,118.546875
02/01/1979,100.890625
03/01/1979,87.375000
04/01/1979,77.765625
05/01/1979,71.640625
...,...
27/12/2023,233.664062
28/12/2023,450.976562
29/12/2023,764.796875


In [21]:
csv_file_name = os.path.join(out_dir, f"all_stations/glofas_reanalysis_complete_series.csv")
pd.concat(all_dfs,axis=1).to_csv(csv_file_name)

In [22]:
csv_file_name = os.path.join(out_dir, f"all_stations/glofas_reanalysis.csv")

df_all = pd.concat(all_dfs,axis=1)
df_all.index = pd.to_datetime(df_all.index,format='%d/%m/%Y')
df_all[df_all.index>='01/01/2003'].to_csv(csv_file_name)

### get correlation of observed data with glofas

In [25]:
df_obs = pd.read_csv(directory / 'data/observations/gauging_stations/all_stations/observations.csv')
df_obs = df_obs.rename(columns={'Unnamed: 0':'date'})
df_obs["date"] = pd.to_datetime(df_obs["date"], format='mixed')
df_obs = df_obs.set_index('date')
df_obs

,Limpopo_em_Mapai,Limpopo__Combomune,Limpopo_em_Chokwe,Limpopo_em_Sicacate,Limpopo_em_Mabalane,Limpopo_em_XaiXai,Changane_em_VGomes_da_Costa,Changane_em_Chibuto,Limpopo_em_Macaretane,Elefantes_em_Munhamane,Singuezi_em_Bingo
date,,,,,,,,,,,
2003-01-01,NaN,1.50,0.88,2.150000,0.930000,1.305,NaN,NaN,96.870000,NaN,NaN
2003-01-02,NaN,1.51,0.89,2.160000,0.926667,1.150,NaN,NaN,96.880000,NaN,NaN
2003-01-03,NaN,1.52,0.89,2.170000,0.920000,1.330,NaN,NaN,96.903333,NaN,NaN
2003-01-04,NaN,1.53,0.89,2.166667,0.920000,1.610,NaN,NaN,96.906667,NaN,NaN
2003-01-05,NaN,1.54,0.89,2.170000,0.910000,1.710,NaN,NaN,96.920000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_all[df_all.index>='01/01/2003'].corrwith(df_obs)

Limpopo_em_Chokwe              0.510731
Changane_em_Chibuto                 NaN
Changane_em_VGomes_da_Costa         NaN
Elefantes_em_Munhamane              NaN
Limpopo__Combomune                  NaN
Limpopo_em_Mabalane                 NaN
Limpopo_em_Macaretane               NaN
Limpopo_em_Mapai                    NaN
Limpopo_em_Sicacate                 NaN
Limpopo_em_XaiXai                   NaN
Singuezi_em_Bingo                   NaN
dtype: float64